In [1]:
import sys
sys.path.append('../')

In [2]:
from scripts.utils import data_path
from scripts.cross_match_scripts import pandas_to_fits, desi_reliable_magnitudes
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
import seaborn as sns
%matplotlib inline

# Counterparts and field sources catalogs construction

## Load DESI-CSC 30 arcsec cone search match

In [3]:
desi_csc_orig = pd.read_pickle(data_path+'/csc_allsky_desi_r30_gaia_dered.gz_pkl',compression='gzip')

In [5]:
desi_csc_orig = desi_reliable_magnitudes(desi_csc_orig) #make cuts for SNR of flux + add colors
desi_csc_orig

/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getat

,csc_name,csc_ra,csc_dec,csc_flux_05_2,csc_radec_err,csc_r_98,release,objid,brickid,ra,dec,brick_primary,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,dist_arcsec,desi_id,all_mag_g,rel_mag_g,rel_dered_mag_g,all_mag_r,rel_mag_r,rel_dered_mag_r,all_mag_z,rel_mag_z,rel_dered_mag_z,all_mag_w1,rel_mag_w1,rel_dered_mag_w1,vega_mag_w1,all_mag_w2,rel_mag_w2,rel_dered_mag_w2,vega_mag_w2,all_mag_w3,rel_mag_w3,rel_dered_mag_w3,vega_mag_w3,all_mag_w4,rel_mag_w4,rel_dered_mag_w4,vega_mag_w4,rel_desi_flux_corr_z,rel_dered_g_r,rel_dered_g_z,rel_dered_r_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4
0,2CXO J000000.0+004331,0.000020,0.725285,2.625836e-15,0.492084,1.376434,9010,9135,336127,359.993680,0.721193,1,0.264035,0.761373,1.464151,-2.279967,-0.936121,-109.650260,-1274.12740,2924.99000,2268.87720,293.554100,3.076859,0.628511,0.000788,0.000010,23.857758,22.736668,22.052845,NaN,NaN,NaN,NaN,14.279862,36.266293,25.085913,-3.999284,-0.742145,-3.078044,-4.096002,0.0,0.0,0.0,0.0,0.0,0.0,27.164009,9010_336127_9135,23.945846,23.945846,23.857758,22.796006,22.796006,22.736668,22.086035,22.086035,22.052845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.520281e-15,1.121090,1.804913,0.683823,NaN,NaN,NaN,NaN,NaN,NaN
1,2CXO J000000.0+004331,0.000020,0.725285,2.625836e-15,0.492084,1.376434,9010,9169,336127,359.994202,0.722597,1,0.994499,2.786103,5.748118,6.166792,6.704570,-0.933517,1110.99740,2478.61230,1837.17550,249.912500,2.884505,0.622227,0.000782,0.000011,22.417845,21.328133,20.567974,20.519806,20.430973,NaN,14.885468,49.511795,119.418724,90.869820,10.473576,5.288656,-0.026110,3.618951,0.0,0.0,0.0,0.0,0.0,0.0,23.070881,9010_336127_9169,22.505989,22.505989,22.417845,21.387507,21.387507,21.328133,20.601186,20.601186,20.567974,20.524852,20.524852,20.519806,17.825852,20.434073,20.434073,20.430973,17.095073,NaN,NaN,NaN,NaN,14.885717,NaN,NaN,NaN,9.894579e-15,1.089712,1.849871,0.760159,0.048168,0.897160,0.088833,NaN,NaN,NaN
2,2CXO J000000.0+004331,0.000020,0.725285,2.625836e-15,0.492084,1.376434,9010,9295,336127,359.997367,0.721166,1,0.287270,0.334849,0.528142,-1.608118,0.423848,16.828413,692.35394,2346.16870,1762.25840,213.479600,2.978809,0.614420,0.000778,0.000010,23.765974,23.628395,23.159851,NaN,23.428870,19.434230,15.398930,13.914554,14.056722,7.716652,-2.775487,0.332233,0.469412,2.225948,0.0,0.0,0.0,0.0,0.0,0.0,17.638166,9010_336127_9295,23.854274,23.854274,23.765974,23.687877,23.687877,23.628395,23.193123,23.193123,23.159851,NaN,NaN,NaN,NaN,23.431975,NaN,NaN,NaN,19.434892,NaN,NaN,NaN,15.399180,NaN,NaN,NaN,9.091730e-16,0.137579,0.606123,0.468544,NaN,NaN,NaN,NaN,NaN,NaN
3,2CXO J000000.0+004331,0.000020,0.725285,2.625836e-15,0.492084,1.376434,9010,9271,336127,359.996619,0.722544,1,0.195305,0.580013,1.060394,-1.374197,0.465793,55.299458,840.05010,1941.29630,1459.10840,181.554780,2.827259,0.595737,0.000749,0.000010,24.184935,23.031937,22.403067,NaN,23.326414,18.142536,15.188987,8.605151,22.155508,14.287992,-2.310637,0.359518,1.513426,2.704489,0.0,0.0,0.0,0.0,0.0,0.0,15.723989,9010_336127_9271,24.273217,24.273217,24.184935,23.091406,23.091406,23.031937,22.436332,22.436332,22.403067,NaN,NaN,NaN,NaN,23.329518,NaN,NaN,NaN,18.143198,NaN,NaN,NaN,15.189237,NaN,NaN,NaN,1.825409e-15,1.152998,1.781868,0.628870,NaN,NaN,NaN,NaN,NaN,NaN
4,2CXO J000000.0+004331,0.000020,0.725285,2.625836e-15,0.492084,1.376434,9010,9360,336127,359.998741,0.722629,1,0.162211,0.722331,4.141845,10.932156,10.346854,281.147600,-6404.08400,1451.76900,1055.47450,125.596010,2.283064,0.530093,0.000696,0.000010,24.386390,22.793606,20.923704,19.898174,19.959871,16.377000,NaN,6.180582,23.467144,46.417503,16.518280,7.533284,7.418694,-20.397125,0.0,0.0,0.0,

### Primary filters and DESI duplicates (ra,dec) removal

In [6]:
#step 1: filter brick ID
#next sort by name_csc, ra, dec and flux_g, with flux_g from largest to smallest
desi_csc = desi_csc_orig.query('brick_primary == True')
print('desi reliable catalog:', len(desi_csc))
desi_csc = desi_csc.sort_values(by=['csc_name', 'csc_ra', 'csc_dec', 'flux_g'], ascending=[True, True, True, False])

#step 2: remove duplicates in ['ra_csc', 'dec_csc', 'ra', 'dec'] and keep the one with the highest flux_g 
desi_csc = desi_csc.drop_duplicates(subset=['csc_ra', 'csc_dec', 'ra', 'dec'], keep='first')

print('desi reliable catalog after non-primary brick and duplicates removal:', len(desi_csc))

#sort by sep_csc so that the closest objects are first
desi_csc.sort_values(by=['csc_name', 'dist_arcsec'], inplace=True)
desi_csc.sample(10)

desi reliable catalog: 3249397
desi reliable catalog after non-primary brick and duplicates removal: 3249392


,csc_name,csc_ra,csc_dec,csc_flux_05_2,csc_radec_err,csc_r_98,release,objid,brickid,ra,dec,brick_primary,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,dist_arcsec,desi_id,all_mag_g,rel_mag_g,rel_dered_mag_g,all_mag_r,rel_mag_r,rel_dered_mag_r,all_mag_z,rel_mag_z,rel_dered_mag_z,all_mag_w1,rel_mag_w1,rel_dered_mag_w1,vega_mag_w1,all_mag_w2,rel_mag_w2,rel_dered_mag_w2,vega_mag_w2,all_mag_w3,rel_mag_w3,rel_dered_mag_w3,vega_mag_w3,all_mag_w4,rel_mag_w4,rel_dered_mag_w4,vega_mag_w4,rel_desi_flux_corr_z,rel_dered_g_r,rel_dered_g_z,rel_dered_r_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4
2639618,2CXO J160620.5+000254,241.585420,0.048506,1.050290e-15,0.497858,1.392584,9010,1374,331334,241.586802,0.043922,1,0.165069,0.303691,0.973848,3.448817,3.663282,52.076927,883.409240,776.951800,113.733170,31.875507,3.231070,0.698089,0.000774,0.000010,23.865154,23.396027,22.306210,21.122007,21.069557,18.203957,15.132923,4.601102,3.238732,5.498187,6.199310,3.060736,1.448360,2.839604,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,17.239267,9010_331334_1374,24.455836,24.455836,23.865154,23.793920,NaN,NaN,22.528772,22.528772,22.306210,21.155825,21.155825,21.122007,18.456825,21.090324,NaN,NaN,NaN,18.208387,NaN,NaN,NaN,15.134595,NaN,NaN,NaN,1.995735e-15,NaN,1.558944,NaN,1.184203,NaN,NaN,NaN,NaN,NaN
352176,2CXO J012448.0-091752,21.200128,-9.297955,1.957346e-15,0.887414,2.482230,9010,3469,277371,21.202908,-9.301935,1,1.582945,5.291246,11.747754,6.792634,1.826146,43.416737,821.452300,1275.699700,449.766300,68.192430,3.245163,0.706015,0.000936,0.000011,21.846886,20.587065,19.766918,20.411062,21.840730,18.404700,15.213107,56.537937,112.215126,97.011440,12.236480,1.534413,1.328300,2.770887,0.627974,0.367068,9.186933,0.07898,-6.211823,0.27207,17.404682,9010_277371_3469,22.001335,22.001335,21.846886,20.691105,20.691105,20.587065,19.825113,19.825113,19.766918,20.419904,20.419904,20.411062,17.720904,21.846161,NaN,NaN,NaN,18.405857,NaN,NaN,NaN,15.213544,NaN,NaN,NaN,2.069282e-14,1.259821,2.079968,0.820147,-0.644144,NaN,NaN,NaN,NaN,NaN
999336,2CXO J050634.4-320029,76.643422,-32.008088,1.617979e-15,0.669314,1.872172,9010,5531,155622,76.635169,-32.008746,1,0.071487,0.070903,0.061583,0.253570,-0.438675,4.099005,1025.931800,3869.219000,2266.104000,417.903930,5.229713,1.291200,0.001680,0.000018,25.316729,25.341213,25.508375,23.987026,NaN,20.967947,14.972069,4.446739,3.375226,1.258924,0.579877,-0.498470,0.167996,4.315961,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,25.305052,9010_155622_5531,25.364432,25.364432,25.316729,25.373338,NaN,NaN,25.526348,NaN,NaN,23.989755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.968304,NaN,NaN,NaN,14.972204,14.972204,14.972069,8.352204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
211305,2CXO J005549.4+261612,13.956248,26.270225,8.966642e-16,0.738558,2.065856,9010,776,476630,13.949613,26.266052,1,0.167498,0.280349,0.261832,-0.694677,-3.889341,37.828762,-72.877396,2003.251200,648.522460,131.602230,3.610075,0.738236,0.000997,0.000013,24.253296,23.755001,23.884605,NaN,NaN,18.554045,NaN,7.496820,7.139406,3.003680,-1.319899,-3.341748,1.194156,-0.266481,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,26.162783,9010_476630_776,24.439976,24.439976,24.253296,23.880752,23.880752,23.755001,23.954943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.555445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.498295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2980397,2CXO J215048.3-053932,327.701360,-5.659075,5.002260e-16,0.822334,2.300192,9010,2276,298591,327.706782,-5.656811,1,0.218863,0.161229,0.265631,1.092810,2.867623,-89.139180,-2194.388700,1112.175300,211.527730,60.885490,3.064062,0.613538,0.000632,0.000009,24.029116,24.400250,23.893917,22.396742,21.351960,NaN,

## Calculating source densities and false association radius (r_false) for each CSC source 

In [11]:
def annuli_area_deg2(r_in_arcsec, r_out_arcsec):
    r_in_deg = r_in_arcsec/3600
    r_out_deg = r_out_arcsec/3600
    area = np.pi*(r_out_deg**2 - r_in_deg**2)
    return area

def r_false(desi_rho_deg2, thresh = 0.03):
    ''' Belvedersky+ 2022 '''
    desi_rho_arcsec2 = desi_rho_deg2/(3600**2)
    return np.sqrt(-np.log(1-thresh)/(np.pi*desi_rho_arcsec2))


#make a dataframe of CSC sources
csc = pd.DataFrame(desi_csc.groupby(by = 'csc_name', ).agg({'csc_ra': np.mean, 'csc_dec': np.mean}))


#calculate total number of DESI sources withing 10-30 arcsec of each CSC source
num_src_desi_10_30 =  desi_csc.groupby(by = ['csc_name', pd.cut(desi_csc.dist_arcsec, [10, 30]) ]).size().unstack()


#calculate source densities for 10-30'' annulus
src_dens_deg2_10_30 = num_src_desi_10_30/annuli_area_deg2(10, 30)

#drop sources with  zeros in the 10-30'' source density (very few sources)
csc.drop(csc[src_dens_deg2_10_30 == 0].index, inplace=True)

#assign r_false for 10-30'' annulus density
csc['r_false_003_dens_10_30'] = r_false(src_dens_deg2_10_30)

csc.reset_index(inplace=True)

#assign r_false for each CSC source from csc dataframe to desi_csc dataframe
desi_csc = desi_csc.merge(csc[['csc_name','r_false_003_dens_10_30']], on='csc_name')

desi_csc.sample(10)

,csc_name,csc_ra,csc_dec,csc_flux_05_2,csc_radec_err,csc_r_98,release,objid,brickid,ra,dec,brick_primary,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,dist_arcsec,desi_id,all_mag_g,rel_mag_g,rel_dered_mag_g,all_mag_r,rel_mag_r,rel_dered_mag_r,all_mag_z,rel_mag_z,rel_dered_mag_z,all_mag_w1,rel_mag_w1,rel_dered_mag_w1,vega_mag_w1,all_mag_w2,rel_mag_w2,rel_dered_mag_w2,vega_mag_w2,all_mag_w3,rel_mag_w3,rel_dered_mag_w3,vega_mag_w3,all_mag_w4,rel_mag_w4,rel_dered_mag_w4,vega_mag_w4,rel_desi_flux_corr_z,rel_dered_g_r,rel_dered_g_z,rel_dered_r_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4,r_false_003_dens_10_30
2422341,2CXO J143120.5+395241,217.835573,39.878194,NaN,0.410685,1.148747,9011,2756,542551,217.838392,39.872150,1,0.145236,0.170445,0.530148,2.105322,-2.227575,-9.053241,-2612.953600,894.85690,178.870820,114.554146,4.984376,1.212417,0.001856,0.000022,24.558907,24.396852,23.175478,21.689648,NaN,NaN,NaN,4.344609,2.279572,5.674169,4.700282,-2.452779,-0.390024,-12.243992,0.0,0.0,0.0,0.0,0.0,0.0,23.111303,9011_542551_2756,24.594814,24.594814,24.558907,24.421039,NaN,NaN,23.189007,23.189007,23.175478,21.691704,21.691704,21.689648,18.992704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.961810e-16,NaN,1.383429,NaN,1.485830,NaN,NaN,NaN,NaN,NaN,1.488360
1985871,2CXO J123139.9+015810,187.916474,1.969605,1.694646e-14,2.477891,6.931032,9010,3979,342639,187.913447,1.963809,1,0.500935,0.998716,1.730858,1.037480,2.672111,68.998800,-1239.888200,1356.95680,601.159300,66.268590,3.066492,0.586123,0.000640,0.000009,23.192184,22.462082,21.882357,22.456709,21.430813,17.902458,NaN,18.452879,24.487062,14.090137,1.816773,2.045732,1.746122,-3.623405,0.0,0.0,0.0,0.0,0.0,0.0,23.539714,9010_342639_3979,23.250547,23.250547,23.192184,22.501395,22.501395,22.462082,21.904346,21.904346,21.882357,22.460051,NaN,NaN,NaN,21.432864,NaN,NaN,NaN,17.902896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.948792e-15,0.730102,1.309827,0.579725,NaN,NaN,NaN,NaN,NaN,NaN,1.077196
2854249,2CXO J201204.5-573136,303.018770,-57.526940,6.151835e-15,0.861411,2.409496,9010,10711,52454,303.012501,-57.529917,1,0.224299,0.417597,0.707336,2.578058,4.236895,93.721300,13435.504000,2904.14580,2080.679700,169.628680,3.673169,0.934622,0.001092,0.000014,23.945461,23.328560,22.809067,21.461609,20.926142,17.569073,12.178863,12.087522,19.048483,9.212456,4.940979,4.096055,3.097067,49.942276,0.0,0.0,0.0,0.0,0.0,0.0,16.176913,9010_52454_10711,24.122932,24.122932,23.945461,23.448107,23.448107,23.328560,22.875936,22.875936,22.809067,21.471768,21.471768,21.461609,18.772768,20.932381,20.932381,20.926142,17.593381,17.570404,NaN,NaN,NaN,12.179365,12.179365,12.178863,5.559365,1.255915e-15,0.616901,1.136394,0.519493,1.347458,2.402418,0.535467,NaN,11.149697,NaN,0.834392
359729,2CXO J012745.6-061936,21.940156,-6.326937,2.687901e-15,0.437677,1.224247,9010,2389,294507,21.946412,-6.326657,1,0.074752,0.127044,-0.058841,-1.880150,-0.551579,35.727463,-56.807327,2464.72730,1606.348400,140.973400,3.155410,0.645779,0.000676,0.000009,25.175120,24.645258,NaN,NaN,NaN,18.616438,NaN,3.711145,5.091819,-0.698634,-3.339800,-0.443251,0.929015,-0.170866,0.0,0.0,0.0,0.0,0.0,0.0,22.408627,9010_294507_2389,25.315943,NaN,NaN,24.740115,24.740115,24.645258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.617495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.163504
2691373,2CXO J163612.9+823331,249.053891,82.558768,3.070151e-15,0.978815,2.737892,9011,2862,659266,249.067465,82.555955,1,0.149746,0.809916,2.378249,11.517674,8.105788,31.724573,266.233150,663.89270,203.146580,49.810955,3.902251,1.762811,0.004518,0.000053,24.281603,22.540283,21.453854,19.830559,20.218166,18.744411,16.436052,3.858382,11.54369

In [12]:
#calculate for each CSC source the number of DESI sources within r_false
tmp_df = pd.DataFrame(desi_csc.query('dist_arcsec <= r_false_003_dens_10_30').groupby(by = 'csc_name').dist_arcsec.apply('count'))
print(tmp_df.dist_arcsec.value_counts())
tmp_df.columns = ['num_srcs_sep_less_r_false']

#merge with desi_csc dataframe
desi_csc = desi_csc.merge(tmp_df, on='csc_name')

1    73359
2     1883
3       41
4        1
Name: dist_arcsec, dtype: int64


## Counterpart  assignment

In [13]:
desi_csc_ctps = desi_csc.query('dist_arcsec <= r_false_003_dens_10_30 & num_srcs_sep_less_r_false==1 & dist_arcsec < csc_r_98 ')

In [14]:
print('delete ', desi_csc_ctps.duplicated(subset='desi_id').sum(), ' duplicated desi_id')
desi_csc_ctps = desi_csc_ctps.drop_duplicates(subset='desi_id')
print('Final number of counterparts:', len(desi_csc_ctps))
desi_csc_ctps['is_counterpart'] = True
#dublicates will appear here if untreated
#desi_csc_ctps.pivot_table(values='name_csc', index='desi_id', aggfunc='count').sort_values(by = 'name_csc',ascending=False)

delete  1  duplicated desi_id
Final number of counterparts: 73193


## Field source assignment

In [15]:
desi_csc_field = desi_csc.query("dist_arcsec>=10 & dist_arcsec<=50 ")
print('number of possible field sources: ', len(desi_csc_field))

number of possible field sources:  1424435


In [16]:
tmp_df = desi_csc_field.groupby(by = 'desi_id', ).agg({'csc_name': 'count'})
clear_field_ids = tmp_df.index[tmp_df['csc_name']==1]
desi_csc_field = desi_csc_field[desi_csc_field.desi_id.isin(clear_field_ids)==True]
fraction_to_retain = 0.3
desi_csc_field = desi_csc_field.sample(frac=fraction_to_retain)
desi_csc_field['is_counterpart'] = False
desi_csc_field['csc_flux_05_2'] = np.nan
desi_csc_field['csc_r_98'] = np.nan

print(f'Final number of secure field sources  ({fraction_to_retain} of it): ', len(desi_csc_field))

Final number of secure field sources  (0.3 of it):  372584


## Saving the combined catalogs

In [17]:
desi_csc_training = pd.concat([desi_csc_ctps, desi_csc_field])
#save cat to pickle
desi_csc_training.to_pickle(data_path+'csc_allsky_desi_r30_gaia_dered_training.gz_pkl', compression='gzip')